In [ ]:
seed = 43
use_maf = False

# simulation setup
setup_opts = {        
    'n_hiddens': [50, 50],
    'reg_lambda': 0.01,
    'pilot_samples': 1000,
    'verbose': True,
    'prior_norm': False,    
    'svi': False,
    'n_rnn': 5,
}


run_opts = {
    'n_train': 1000,
    'n_rounds': 10,
    'minibatch': 100,
    'epochs': 500,
    'moo': 'resample',
    'proposal': 'discrete',
    'n_null': 10,
    'train_on_all': True
}
  
if run_opts['train_on_all']:
    run_opts['epochs'] = [run_opts['epochs'] // (r+1) for r in range(run_opts['n_rounds'])]

if use_maf:
    # control MAF seed
    rng_maf = np.random
    rng_maf.seed(seed)
    
    setup_opts.update({
        'mode': 'random',
        'n_mades': 5,
        'rng': rng_maf,
        'act_fun': 'tanh',
        'batch_norm': False
    })

pars_true = np.array([-4.60517019, -0.69314718,  0.        , -4.60517019])  # from SNL paper
obs_stats = np.array([[9.52251656e+01, 5.19668874e+01, 8.67364561e+00, 8.05149566e+00,
        9.80002316e-01, 9.34987651e-01, 9.77287472e-01, 9.26921876e-01,
        2.29361846e-02]])  # de-normalized, from SNL paper

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
import timeit
from copy import deepcopy

from delfi.utils.viz import plot_pdf
import delfi.inference as infer
import delfi.distribution as dd
import delfi.generator
from delfi.summarystats import Identity

from lfimodels.snl_exps.util import save_results, load_results, StubbornGenerator, stubborn_defaultrej
from lfimodels.snl_exps.util import init_g_lv as init_g
from lfimodels.snl_exps.util import load_setup_lv as load_setup
from lfimodels.snl_exps.util import load_gt_lv as load_gt
from lfimodels.snl_exps.util import calc_all_lprob_errs
from lfimodels.snl_exps.LotkaVolterra import LotkaVolterra, LotkaVolterraStats
import snl.simulators.lotka_volterra as sim_lv

model_id = 'lv'
save_path = 'results/' + model_id
exp_id = 'seed'+str(seed)

print('pars_true : ', pars_true)
print('obs_stats : ', obs_stats)



In [ ]:
model = LotkaVolterra(dt=1.0, duration=5.0)
prior = dd.Uniform(lower= [-5,-5,-5,-5], upper = [2,2,2,2])
summary = Identity()
#Identity(idx = np.arange(10))
g = StubbornGenerator(model=model, prior=prior, summary=summary)

#x0 was generate by simulation using the parameters from the SNL papers, initial state removed
x0 = np.array([90., 145., 196.,  81., 181.,  25., 113.,  10.,  78.,
         4.])

In [ ]:
# initialize inference object
res = infer.SNPEC(g, obs=x0, seed=seed, **setup_opts)

In [ ]:
# train
t = timeit.time.time()
logs_C, tds_C, posteriors_C = res.run(**run_opts)
print('fitting time : ', timeit.time.time() - t)

In [ ]:
for r in range(run_opts['n_rounds']):
    plt.plot(logs_C[r]['loss'])
    plt.show()

for r in range(len(logs_C)):
    
    posterior_C = posteriors_C[r]
    #posterior_C.ndim = posterior_A.ndim
    
    g2 = deepcopy(g)
    g2.proposal = posterior_C
    samples = np.array(g2.draw_params(5000)) 
    
    fig,_ = plot_pdf(dd.Gaussian(m=0.00000123*np.ones(pars_true.size), S=1e-30*np.eye(pars_true.size)), 
                   samples=samples.T,
                   gt=pars_true, 
                   lims=[[-5,2],[-5,2],[-5,2],[-5,2]],
                   #lims=[0,10],
                   resolution=100,
                   ticks=True,
                   figsize=(16,16));
    
    fig.suptitle('SNPE-C posterior estimates, round r = '+str(r+1), fontsize=14)
        
    plt.figure()
    plt.plot(np.arange(run_opts['n_rounds']) + 1, [-posterior_C.eval(pars_true.reshape(1, -1), log=True) for posterior_C in posteriors_C])
    plt.ylabel('log p of true params')
    plt.xlabel('round')

In [ ]:
if False:
    d = model.gen(np.tile(pars_true, (100, 1)))
    y = np.stack([x[0]['data'] for x in d if x[0]['data'] is not None])
    a, b = y[:,::2].T, y[:,1::2].T
    plt.plot(a,'b'); plt.plot(b,'g');